Full code to make all waypoints from road sections 

In [1]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
import glob
import os
try:
    sys.path.append(glob.glob('./carla/dist/carla-0.9.14-py3.7-win-amd64.egg')[0])
except IndexError:
    print('Couldn\'t import Carla egg properly')
import carla

try:
    sys.path.insert(0,r'C:\Users\ashokkumar\source\repos\AD\carla')
except IndexError:
    pass

from carla import ColorConverter as cc
from agents.navigation.behavior_agent import BehaviorAgent   # type: ignore
from agents.navigation.basic_agent import BasicAgent  # type: ignore
from scipy.spatial import KDTree

SECONDS_PER_EPISODE = 25

SECONDS_PER_EPISODE = 10  # seconds per episode maybe needs to be removed later
FIXED_DELTA_SECONDS = 0.02 # seconds per frame
SHOW_PREVIEW = True # Show preview for debugging
NO_RENDERING = False # No rendering for training
SYNCRONOUST_MODE = False # Synchronous mode for training
SPIN = 10   # angle of spin for initail spawn
HEIGHT = 480 # height of image
WIDTH = 640 # width of image
SHOW_CAM = SHOW_PREVIEW     # render camera
front_camera = None
CAMERA_POS_Z = 1.3      # camera position similar to tesla model 3
CAMERA_POS_X = 1.4    # camera position similar to tesla model 3
im_width = WIDTH
im_height = HEIGHT


In [2]:
## INITIALIZE THE SIMULATOR

# connect to the sim 
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
world = client.load_world('Town02')

settings = world.get_settings()
settings.no_rendering_mode = NO_RENDERING
settings.synchronous_mode = SYNCRONOUST_MODE
settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
world.apply_settings(settings)
blueprint_library = world.get_blueprint_library()
actor_list = []
vehicle = None
collision_hist = []

# get map look at the map
if SHOW_CAM:
    spectator = world.get_spectator()

town_map = world.get_map()
roads = town_map.get_topology()



In [3]:
# Define the waypoint and KDTree generation logic

town_map = world.get_map()
waypoints = town_map.generate_waypoints(4)
waypoint_positions = np.array([(wp.transform.location.x, wp.transform.location.y) for wp in waypoints])
waypoint_tree = KDTree(waypoint_positions)
spawn_position = None        
start_waypoint = None
dest_waypoint = None
observation = None

print(f"Generated {len(waypoints)} waypoints")
# draw all point in the sim for 60 seconds
for wp in waypoints:
    world.debug.draw_string(wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=20.0,
        persistent_lines=True)



Generated 770 waypoints


In [4]:
## CLEANUP THE SIMULATOR
def cleanup():
        for actor in actor_list:
            if actor.is_alive:
                try:
                    actor.destroy()
                except Exception as e:
                    print(f"Failed to destroy actor {actor.id}: {e}")


In [5]:
## SPAWN THE VEHICLE
def spawn_vehicle():
    spawn_points = world.get_map().get_spawn_points()
    transform = random.choice(spawn_points)
    blueprint_library = world.get_blueprint_library()
    model_3 = blueprint_library.find('vehicle.tesla.model3')
    vehicle = None
    while vehicle is None:
        try:
            vehicle = world.spawn_actor(model_3, transform)
        except Exception as e:
            print(f"Failed to spawn vehicle: {e}")
            time.sleep(0.2)  # Brief delay before retrying
    return vehicle

In [6]:
def initial_waypoint(initial_pos):
    # Query the KD-tree for the closest waypoint to initial_pos
    _, closest_index = waypoint_tree.query([initial_pos.x, initial_pos.y])
    closest_waypoint = waypoints[closest_index]
    return closest_waypoint

def next_waypoint(start_waypoint, look_ahead=5.0):
    next_wps = start_waypoint.next(look_ahead)
    if next_wps:
        return next_wps[0]
    else:
        # Handle the case where no next waypoint is found
        print("No next waypoint found")
        return None
    
def draw_waypoint(waypoint, color=carla.Color(255, 0, 0), lt=20.0):
    world.debug.draw_string(waypoint.transform.location, 'O', draw_shadow=False,
        color=color, life_time=lt, persistent_lines=True)
    
def vector3d_to_location(vector):
    return carla.Location(x=vector.x, y=vector.y, z=vector.z)

def draw_arrow(start_location, end_location,color=carla.Color(255, 0, 0), lt=20.0):
    if start_location and end_location:
        start_location = vector3d_to_location(start_location)
        end_location = vector3d_to_location(end_location)
        end_location += carla.Location(z=2)
        start_location += carla.Location(z=2)
        world.debug.draw_arrow(start_location, end_location, thickness=0.1, arrow_size=0.3, color=color, life_time=lt)

def draw_heading_arrow(location, yaw, length=5.0, color=carla.Color(255, 0, 0), life_time=5.0):
    yaw_corrected = (yaw) % 360

    # Convert yaw from degrees to radians
    yaw_rad = np.deg2rad(yaw_corrected)

    # Compute the direction vector based on the corrected yaw (heading)
    direction_x = np.cos(yaw_rad)
    direction_y = np.sin(yaw_rad)

    # Calculate the end location of the arrow using the direction vector
    end_location = carla.Location(
        x=location.x + length * direction_x,
        y=location.y + length * direction_y,
        z=2
    )
    location.z = 2
    # Draw the arrow in the CARLA world
    world.debug.draw_arrow(
        location,                 # Start point (the given location)
        end_location,             # End point (direction of the yaw)
        thickness=0.1,            # Arrow thickness
        arrow_size=0.3,           # Arrowhead size
        color=color,              # Color of the arrow
        life_time=life_time       # How long the arrow will stay in the world (in seconds)
    )

def draw_relative_heading_arrow(vehicle_transform, waypoint_transform, length=5.0, color=carla.Color(0, 255, 0), life_time=5.0):
# Extract vehicle and waypoint yaw in degrees
    vehicle_yaw = vehicle_transform.rotation.yaw
    waypoint_yaw = waypoint_transform.rotation.yaw

    # Calculate the relative heading (difference between vehicle and waypoint yaw)
    relative_heading = (vehicle_yaw - waypoint_yaw) % 360
    if relative_heading > 180:
        relative_heading -= 360

    # The correct heading to draw the arrow is the vehicle's global yaw minus the relative heading
    absolute_heading = vehicle_yaw - relative_heading

    # Convert the absolute heading to radians
    absolute_heading_rad = np.deg2rad(absolute_heading)

    # Get the vehicle's current location
    vehicle_location = vehicle_transform.location

    # Compute the arrow's end location based on the absolute heading direction
    arrow_end_location = carla.Location(
        x=vehicle_location.x + length * np.cos(absolute_heading_rad),
        y=vehicle_location.y + length * np.sin(absolute_heading_rad),
        z=vehicle_location.z
    )

    # Draw an arrow in the CARLA world from the vehicle location in the direction of absolute heading
    world.debug.draw_arrow(
        vehicle_location,             # Start point (vehicle's current location)
        arrow_end_location,           # End point (direction of the relative heading)
        thickness=0.1,                # Arrow thickness
        arrow_size=0.3,               # Arrowhead size
        color=color,                  # Color of the arrow
        life_time=life_time           # How long the arrow will stay in the world (in seconds)
    )




In [7]:
## OBSERVATION LOGIC - 1
def get_speed(v, max_speed=50.0):
    if not isinstance(v, carla.Vector3D):
        print("Invalid velocity vector provided.")
        return 0
    speed = int(3.6 * np.sqrt(v.x**2 + v.y**2 + v.z**2))
    normalized_speed = np.clip(speed / max_speed, -1, 1)  # Normalize speed to [-1, 1]
    return normalized_speed

def get_relative_heading(vehicle_transform, waypoint):
        # Extract vehicle and waypoint yaw in degrees
        vehicle_yaw = vehicle_transform.rotation.yaw
        waypoint_yaw = waypoint.transform.rotation.yaw
        num_lookahead = 5  # Number of waypoints to look ahead
        distance_lookahead = 3.0
        yaw_differences_sum = 0  # Accumulator for the sum of yaw differences
        current_waypoint = waypoint  # Start from the current waypoint
        turn_direction = turn_directon(vehicle_transform, waypoint, num_lookahead, distance_lookahead)
        
        for i in range(1, num_lookahead + 1):
            # Fetch the next waypoint 'i' steps ahead
            future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
            
            if not future_waypoint:
                break  # Stop if there's no more valid waypoints
        
            # Check if the current waypoint is part of a turn
            future_yaw = future_waypoint.transform.rotation.yaw
            yaw_diff = abs(current_waypoint.transform.rotation.yaw - future_yaw) % 360
            yaw_diff = yaw_diff if yaw_diff <= 180 else 360 - yaw_diff
            
            # Accumulate the yaw difference
            yaw_differences_sum += yaw_diff
            
            # Move to the next waypoint for the next iteration
            current_waypoint = future_waypoint
            
            # If we gathered yaw differences, process the sum
        if yaw_differences_sum > 22.5:
            # Calculate relative heading as the difference between the yaw sum and the vehicle's yaw
            relative_heading = (yaw_differences_sum) % 360
            
            # Normalize to [-180, 180] range
            if relative_heading > 180:
                relative_heading -= 360

            # Adjust the heading based on the turn direction
            normalized_heading = turn_direction * (relative_heading / 180.0)
            return np.clip(normalized_heading, -1, 1)

        # If no turn detected, calculate the basic relative heading from the closest waypoint
        relative_heading = (waypoint_yaw - vehicle_yaw) % 360

        # Normalize to [-180, 180] range
        if relative_heading > 180:
            relative_heading -= 360

        normalized_heading = relative_heading / 180.0
        return np.clip(normalized_heading, -1, 1)


def turn_directon(vehicle_transform, waypoint, num_lookahead=5, distance_lookahead=3.0):
    # Extract yaw angles from waypoints
    vehicle_yaw = vehicle_transform.rotation.yaw  # Vehicle's current yaw
    current_waypoint = waypoint  # Start from the current waypoint
    future_waypoint = None
    for i in range(1, num_lookahead + 1):
        future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
        if not future_waypoint:
            break  # Stop if there's no valid future waypoint
        current_waypoint = future_waypoint  # Move to the next waypoint

    if future_waypoint:
        # Get the yaw of the future waypoint
        future_yaw = future_waypoint.transform.rotation.yaw
        
        # Calculate the yaw difference between the vehicle and the future waypoint
        yaw_diff = (future_yaw - vehicle_yaw) % 360
        
        # Normalize the yaw difference to the [-180, 180] range
        if yaw_diff > 180:
            yaw_diff -= 360
        if yaw_diff > 5:  # If the future yaw is greater than the vehicle's yaw by a threshold
            return 1
        else:
            return -1

In [8]:
def get_relative_heading_new(vehicle_transform, waypoint, num_lookahead=5, distance_lookahead=3):
    """
    Calculate the relative heading using the sum of yaw differences with multi-waypoint lookahead.
    
    Args:
    - vehicle_transform: The current transform of the vehicle (including yaw).
    - waypoint_transform: The transform of the nearest waypoint.
    - num_lookahead: The number of waypoints to look ahead.
    - distance_lookahead: Distance in meters between each consecutive waypoint.
    
    Returns:
    - Normalized relative heading in the range [-1, 1], where -1 is a left turn, and 1 is a right turn.
    """
    
    # Extract vehicle and waypoint yaw in degrees
    vehicle_yaw = vehicle_transform.rotation.yaw
    waypoint_yaw = waypoint.transform.rotation.yaw
    
    # List to store yaw values of waypoints ahead
    yaw_differences_sum = 0  # Accumulator for the sum of yaw differences
    
    current_waypoint = waypoint  # Start from the current waypoint
    
    for i in range(1, num_lookahead + 1):
        # Fetch the next waypoint 'i' steps ahead
        future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
        
        if not future_waypoint:
            break  # Stop if there's no more valid waypoints
        
        # Calculate yaw difference between current waypoint and future waypoint
        future_yaw = future_waypoint.transform.rotation.yaw
        yaw_diff = abs(current_waypoint.transform.rotation.yaw - future_yaw) % 360
        yaw_diff = yaw_diff if yaw_diff <= 180 else 360 - yaw_diff
        
        # Accumulate the yaw difference
        yaw_differences_sum += yaw_diff
        
        # Move to the next waypoint for the next iteration
        current_waypoint = future_waypoint
    
    # If we gathered yaw differences, process the sum
    if yaw_differences_sum > 22.5:
        # Determine if it's a left or right turn based on the sum
        if yaw_differences_sum > vehicle_yaw:
            turn_direction = 1.0  # Right turn
        else:
            turn_direction = -1.0  # Left turn

        # Calculate relative heading as the difference between the yaw sum and the vehicle's yaw
        relative_heading = (yaw_differences_sum - vehicle_yaw) % 360
        
        # Normalize to [-180, 180] range
        if relative_heading > 180:
            relative_heading -= 360

        # Adjust the heading based on the turn direction
        normalized_heading = turn_direction * (relative_heading / 180.0)
        return np.clip(normalized_heading, -1, 1)

    # If no turn detected, calculate the basic relative heading from the closest waypoint
    relative_heading = (waypoint_yaw - vehicle_yaw) % 360

    # Normalize to [-180, 180] range
    if relative_heading > 180:
        relative_heading -= 360

    normalized_heading = relative_heading / 180.0
    return np.clip(normalized_heading, -1, 1)


In [9]:
   
def get_lateral_distance(vehicle_transform, waypoint, max_distance=3.0):
    # Get the vehicle and waypoint positions
    waypoint_location = waypoint.transform.location
    waypoint_forward_vector = waypoint.transform.get_forward_vector()
    vehicle_location = vehicle_transform.location

    # Convert CARLA location objects to NumPy arrays for easier vector calculations
    waypoint_location_np = np.array([waypoint_location.x, waypoint_location.y])
    waypoint_forward_np = np.array([waypoint_forward_vector.x, waypoint_forward_vector.y])
    vehicle_location_np = np.array([vehicle_location.x, vehicle_location.y])

    # Normalize the waypoint's forward vector (this is the direction of the lane)
    norm = np.linalg.norm(waypoint_forward_np)
    if norm == 0:
        return 0  # Prevent division by zero if the forward vector length is zero
    waypoint_forward_np = waypoint_forward_np / norm

    # Vector from the waypoint (center of the lane) to the vehicle
    vehicle_vector_np = vehicle_location_np - waypoint_location_np

    # Project the vehicle vector onto the waypoint's forward vector (lane direction)
    projection_length = np.dot(vehicle_vector_np, waypoint_forward_np)
    projection_point = projection_length * waypoint_forward_np

    # Calculate the lateral distance between the vehicle and the center line
    lateral_vector = vehicle_vector_np - projection_point
    lateral_distance = np.linalg.norm(lateral_vector)

    # Normalize the lateral distance to the range [-1, 1]
    normalized_lateral_distance = np.clip(lateral_distance / max_distance, -1, 1)

    # Check if the vehicle is to the left or right of the lane center by using a cross product
    cross_product = np.cross(waypoint_forward_np, vehicle_vector_np)
    if cross_product < 0:
        # If the cross product is negative, the vehicle is to the right of the lane
        normalized_lateral_distance *= -1

    return normalized_lateral_distance

In [10]:
## OBSERVATION LOGIC - 2

def get_observation(vehicle_transform, dest_waypoint):
        if vehicle and vehicle.is_alive:
                velocity = vehicle.get_velocity()
                speed = get_speed(velocity)
        else:
                speed = 0  # Default speed if vehicle is not available
        lateral_distance = get_lateral_distance(vehicle_transform, dest_waypoint)
        heading = get_relative_heading(vehicle_transform, dest_waypoint)
        
        print(f"lateral_distance: {lateral_distance}, speed: {speed}, heading: {heading}")
        obs = np.array([
                lateral_distance,
                speed,
                heading,
        ], dtype=np.float32)
        return obs

In [11]:
## RESET THE SIMULATOR
cleanup()
vehicle = spawn_vehicle()
actor_list.append(vehicle)
world.tick()
vehicle_transform = vehicle.get_transform()
start_waypoint = initial_waypoint(vehicle_transform.location)

print("Vehicle spawned at location ", vehicle_transform.location)

spectator.set_transform(carla.Transform(vehicle_transform.location + carla.Location(z=40), carla.Rotation(pitch=-90)))

Vehicle spawned at location  Location(x=193.779999, y=171.289993, z=0.490200)


In [12]:
vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0))
speed = get_speed(vehicle.get_velocity())
print (vehicle.get_velocity())
print("Speed: ", speed)
vehicle_transform = vehicle.get_transform()
start_waypoint = initial_waypoint(vehicle_transform.location)
vehicle.set_transform(start_waypoint.transform)
world.tick()
print("Vehicle moved to location ", vehicle.get_transform().location)
vehicle_transform = vehicle.get_transform()
print(vehicle_transform)

Vector3D(x=0.000000, y=0.000000, z=-0.392000)
Speed:  0.02
Vehicle moved to location  Location(x=193.682907, y=173.089035, z=0.170704)
Transform(Location(x=193.682907, y=173.089035, z=0.170704), Rotation(pitch=0.073390, yaw=-90.013084, roll=-0.193359))


In [13]:
world.wait_for_tick()
dest_waypoint = next_waypoint(start_waypoint, look_ahead=3.0)
print("Start waypoint: ", start_waypoint.transform)    
print("Dest waypoint: ", dest_waypoint.transform)
print(vehicle_transform)
draw_waypoint(dest_waypoint, carla.Color(255, 0, 0))
draw_waypoint(start_waypoint, carla.Color(0, 255, 0))

Start waypoint:  Transform(Location(x=193.682434, y=173.089539, z=0.000000), Rotation(pitch=0.000000, yaw=-90.012947, roll=0.000000))
Dest waypoint:  Transform(Location(x=193.681763, y=170.089539, z=0.000000), Rotation(pitch=0.000000, yaw=-90.012947, roll=0.000000))
Transform(Location(x=193.682907, y=173.089035, z=0.170704), Rotation(pitch=0.073390, yaw=-90.013084, roll=-0.193359))


In [14]:
print(get_observation(vehicle_transform, dest_waypoint))


lateral_distance: 0.0001555623291109539, speed: 0.12, heading: 7.62939453125e-07
[1.5556233e-04 1.2000000e-01 7.6293946e-07]


### Understanding Lateral Distance

In [15]:
## Geting inputs and coverting to numpy arrays
waypoint_location = dest_waypoint.transform.location
waypoint_forward_vector = dest_waypoint.transform.get_forward_vector()
vehicle_location = vehicle_transform.location
print(waypoint_location, waypoint_forward_vector, vehicle_location)
# Convert CARLA location objects to NumPy arrays for easier vector calculations
waypoint_location_np = np.array([waypoint_location.x, waypoint_location.y])
waypoint_forward_np = np.array([waypoint_forward_vector.x, waypoint_forward_vector.y])
vehicle_location_np = np.array([vehicle_location.x, vehicle_location.y])
print(waypoint_location_np, waypoint_forward_np, vehicle_location_np)
# Normalize the waypoint's forward vector (this is the direction of the lane)
norm = np.linalg.norm(waypoint_forward_np)
print(norm)
waypoint_forward_np = waypoint_forward_np / norm
print(waypoint_forward_np)


Location(x=193.681763, y=170.089539, z=0.000000) Vector3D(x=-0.000226, y=-1.000000, z=0.000000) Location(x=193.682907, y=173.089035, z=0.170704)
[193.6817627  170.08953857] [-2.25945318e-04 -1.00000000e+00] [193.6829071  173.08903503]
1.000000025525643
[-2.25945312e-04 -9.99999974e-01]


In [16]:
## 
# Vector from the waypoint (center of the lane) to the vehicle
vehicle_vector_np = vehicle_location_np - waypoint_location_np

# Project the vehicle vector onto the waypoint's forward vector (lane direction)
projection_length = np.dot(vehicle_vector_np, waypoint_forward_np)
projection_point = projection_length * waypoint_forward_np

lateral_vector = vehicle_vector_np - projection_point
lateral_distance = np.linalg.norm(lateral_vector)

# Normalize the lateral distance to the range [-1, 1]
normalized_lateral_distance = np.clip(lateral_distance / 3, -1, 1)

# Check if the vehicle is to the left or right of the lane center by using a cross product
cross_product = np.cross(waypoint_forward_np, vehicle_vector_np)
if cross_product < 0:
    # If the cross product is negative, the vehicle is to the right of the lane
    normalized_lateral_distance *= -1
print(normalized_lateral_distance)


0.0001555623291109539


### Understanding Relative Heading 

In [17]:
# Extract vehicle and waypoint yaw in degrees
waypoint_transform = dest_waypoint.transform
vehicle_yaw = vehicle_transform.rotation.yaw
waypoint_yaw = waypoint_transform.rotation.yaw
print("yaw", vehicle_yaw,"waypoint", waypoint_yaw)
# Look ahead 10 meters (or further until the turn ends)
future_waypoint = next_waypoint(dest_waypoint, 10)  # Look ahead 10 meters
draw_waypoint(future_waypoint, carla.Color(0, 0, 255))
draw_heading_arrow(vehicle_transform.location, vehicle_transform.rotation.yaw,color=carla.Color(0, 255, 0))
draw_heading_arrow(waypoint_transform.location, waypoint_transform.rotation.yaw)

yaw -90.0130844116211 waypoint -90.01294708251953


In [18]:
print(get_observation(vehicle_transform, dest_waypoint))
print(get_relative_heading_new(vehicle_transform, dest_waypoint))


lateral_distance: 0.0001555623291109539, speed: 0.12, heading: 7.62939453125e-07
[1.5556233e-04 1.2000000e-01 7.6293946e-07]
7.62939453125e-07


In [19]:

num_lookahead = 5
distance_lookahead = 3.0
turn_direction = 1
# Extract vehicle and waypoint yaw in degrees
vehicle_yaw = vehicle_transform.rotation.yaw
waypoint_yaw = dest_waypoint.transform.rotation.yaw
draw_heading_arrow(vehicle_transform.location, vehicle_transform.rotation.yaw,color=carla.Color(0, 255, 0))
draw_heading_arrow(dest_waypoint.transform.location, dest_waypoint.transform.rotation.yaw)
# List to store yaw values of waypoints ahead
yaw_differences_sum = 0  # Accumulator for the sum of yaw differences

current_waypoint = dest_waypoint  # Start from the current waypoint


In [20]:

for i in range(1, num_lookahead + 1):
    # Fetch the next waypoint 'i' steps ahead
    future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
    draw_waypoint(future_waypoint, carla.Color(0, 0, 255))
    if not future_waypoint:
        break  # Stop if there's no more valid waypoints
    # Calculate yaw difference between current waypoint and future waypoint
    future_yaw = future_waypoint.transform.rotation.yaw
    draw_heading_arrow(future_waypoint.transform.location, future_yaw)
    yaw_diff = (current_waypoint.transform.rotation.yaw - future_yaw) % 360
    print(yaw_diff)
    if yaw_diff > 180:
        yaw_diff -= 360
    # Accumulate the yaw difference
    yaw_differences_sum += yaw_diff
    # Move to the next waypoint for the next iteration
    current_waypoint = future_waypoint


0.0
0.0
359.90921783447266
0.0
0.0


In [21]:
print(yaw_differences_sum)
# If we gathered yaw differences, process the sum
if abs(yaw_differences_sum) > 25:
    # Calculate relative heading as the difference between the yaw sum and the vehicle's yaw
    relative_heading = (yaw_differences_sum) % 360
    
    # Normalize to [-180, 180] range
    if relative_heading > 180:
        relative_heading -= 360

    # Adjust the heading based on the turn direction
    normalized_heading = turn_direction * (relative_heading / 180.0)
    np.clip(normalized_heading, -1, 1)
    print(normalized_heading)

-0.09078216552734375


In [22]:

# If no turn detected, calculate the basic relative heading from the closest waypoint
relative_heading = (waypoint_yaw - vehicle_yaw) % 360

# Normalize to [-180, 180] range
if relative_heading > 180:
    relative_heading -= 360

normalized_heading = relative_heading / 180.0
np.clip(normalized_heading, -1, 1)
print(normalized_heading)


7.62939453125e-07


## Test

In [23]:
wp = next_waypoint(start_waypoint,45)
draw_waypoint(wp, carla.Color(0, 0, 255))

In [24]:
vehicle.set_transform(wp.transform)
world.wait_for_tick()
vehicle_transform = vehicle.get_transform()
start_waypoint = initial_waypoint(vehicle_transform.location)
spectator.set_transform(carla.Transform(vehicle_transform.location + carla.Location(z=40), carla.Rotation(pitch=-90)))
vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0))
print(vehicle_transform)
dest_waypoint = next_waypoint(start_waypoint, look_ahead=3.0)
print("Start waypoint: ", start_waypoint.transform)    
print("Dest waypoint: ", dest_waypoint.transform)
draw_waypoint(dest_waypoint, carla.Color(255, 0, 0))
draw_waypoint(start_waypoint, carla.Color(0, 255, 0))

Transform(Location(x=193.725723, y=128.092316, z=0.172390), Rotation(pitch=0.066307, yaw=-89.922302, roll=-0.169281))
Start waypoint:  Transform(Location(x=193.725021, y=129.092743, z=0.000000), Rotation(pitch=0.000000, yaw=-89.922165, roll=0.000000))
Dest waypoint:  Transform(Location(x=193.729095, y=126.092751, z=0.000000), Rotation(pitch=0.000000, yaw=-89.922165, roll=0.000000))


In [25]:
print(get_observation(vehicle_transform, dest_waypoint))

lateral_distance: -0.00021861674250962695, speed: 0.18, heading: -0.20441385904947917
[-0.00021862  0.18       -0.20441386]


In [26]:
vehicle_transform.location.y -= 5
vehicle.set_transform(vehicle_transform)


In [27]:
print(get_observation(vehicle_transform, dest_waypoint))

lateral_distance: -0.0024827249148455487, speed: 0.18, heading: -0.20441385904947917
[-0.00248272  0.18       -0.20441386]


: 